In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
import pandas as pd
import numpy as np
from utils import *
import re
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score, confusion_matrix
import numpy as np

In [16]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [17]:
problem = pd.read_csv("../data/для анализа/dataset._problems.csv", sep=";")
normal = pd.read_csv("../data/для анализа/dataset._normal.csv", sep=";")

/var/folders/n9/qf3jg31d5rz3v4ydwp7__zfm0000gn/T/ipykernel_62300/1852484758.py:1: DtypeWarning: Columns (3,11,12,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,37,41,42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  problem = pd.read_csv("data/для анализа/dataset._problems.csv", sep=";")
/var/folders/n9/qf3jg31d5rz3v4ydwp7__zfm0000gn/T/ipykernel_62300/1852484758.py:2: DtypeWarning: Columns (3,12,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,37,41,42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  normal = pd.read_csv("data/для анализа/dataset._normal.csv", sep=";")


In [18]:
problem = pd.get_dummies(problem, columns = ['Сост.пед.сцепл.'], drop_first=True)
normal = pd.get_dummies(normal, columns = ['Сост.пед.сцепл.'], drop_first=True)

In [19]:
normal["Дата и время"] = pd.to_datetime(normal["Дата и время"], dayfirst=True)
problem["Дата и время"] = pd.to_datetime(problem["Дата и время"], dayfirst=True)

In [20]:
normal = nan_handler(normal)
problem = nan_handler(problem)

Regex pattern: Нагрузка\ на\ двигатель|Стояночный\ тормоз|Засоренность\ фильтра\ рулевого\ управления|Засоренность\ фильтра\ навесного\ оборудования|Отопитель|Выход\ блока\ управления\ двигателем|Включение\ тормозков|Засоренность\ фильтра\ слива|Аварийное\ давление\ масла\ КПП|Аварийная\ температура\ масла\ ДВС|Неисправность\ тормозной\ системы|Термостарт|Разрешение\ запуска\ двигателя|Низкий\ уровень\ ОЖ|Аварийная\ температура\ масла\ ГТР|Необходимость\ сервисного\ обслуживания|Подогрев\ топливного\ фильтра|Вода\ в\ топливе|Холодный\ старт|Крутящий\ момент|Положение\ рейки\ ТНВД|Расход\ топлива|Давление\ наддувочного\ воздуха\ двигателя|Температура\ масла\ гидравлики|Педаль\ слива|iButton2
Regex pattern: Нагрузка\ на\ двигатель|Стояночный\ тормоз|Засоренность\ фильтра\ рулевого\ управления|Засоренность\ фильтра\ навесного\ оборудования|Отопитель|Выход\ блока\ управления\ двигателем|Включение\ тормозков|Засоренность\ фильтра\ слива|Аварийное\ давление\ масла\ КПП|Аварийная\ температура

In [21]:
dfs_problem = split_dataframe(problem)
dfs_normal = split_dataframe(normal)

In [22]:
tsfresh_problem = tsfresh_feature(dfs_problem, flag = 1)
tsfresh_normal = tsfresh_feature(dfs_normal, flag = 0)

  0%|          | 0/453 [00:00<?, ?it/s]/Users/alexander/Учёба/ХАКАТОНЫ/EESTech Challenge/tractor_forecasting/utils/utils.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if 'id' not in df.columns:
/Users/alexander/Учёба/ХАКАТОНЫ/EESTech Challenge/tractor_forecasting/utils/utils.py:53: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform 

  0%|          | 1/453 [00:01<11:07,  1.48s/it]/Users/alexander/Учёба/ХАКАТОНЫ/EESTech Challenge/tractor_forecasting/utils/utils.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if 'id' not in df.columns:
/Users/alexander/Учёба/ХАКАТОНЫ/EESTech Challenge/tractor_forecasting/utils/utils.py:53: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to 

In [23]:
concat_df = merge_dataframes_from_lists(tsfresh_problem, tsfresh_normal)

In [28]:
columns_drop = remove_highly_correlated_features_np(concat_df)

/Users/alexander/Учёба/ХАКАТОНЫ/EESTech Challenge/tractor_forecasting/.venv/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/Users/alexander/Учёба/ХАКАТОНЫ/EESTech Challenge/tractor_forecasting/.venv/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [29]:
columns_drop

['Сост.пед.сцепл._Нажат.__mean',
 'Сост.пед.сцепл._Нажат.__length',
 'Сост.пед.сцепл._Нажат.__standard_deviation',
 'Сост.пед.сцепл._Нажат.__variance',
 'Сост.пед.сцепл._Нажат.__root_mean_square',
 'Сост.пед.сцепл._Нажат.__maximum',
 'Сост.пед.сцепл._Нажат.__absolute_maximum',
 'Сост.пед.сцепл._Отпущ.__sum_values',
 'Сост.пед.сцепл._Отпущ.__median',
 'Сост.пед.сцепл._Отпущ.__variance',
 'Сост.пед.сцепл._Отпущ.__root_mean_square',
 'Сост.пед.сцепл._Отпущ.__maximum']

In [49]:
concat_df.shape

(632, 259)

In [34]:
X = concat_df.drop(columns="target")
y = concat_df["target"]

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=666, stratify=y)


In [46]:
def evaluate_model(model, X_train, X_test, y_train, y_test):
    """
    Оценивает модель по различным метрикам.

    Параметры:
        model: обученная модель классификации.
        X_train, X_test: массивы признаков для обучающей и тестовой выборки.
        y_train, y_test: массивы целевой переменной для обучающей и тестовой выборки.
    """
    
    # Предсказание на обучающей выборке
    y_pred_train = model.predict(X_train)
    # Предсказание вероятностей принадлежности к классам на тестовой выборке
    # Берем вероятности для положительного класса
    y_pred_proba_test = model.predict_proba(X_test)[:, 1]
    # Предсказание классов на тестовой выборке
    y_pred_test = model.predict(X_test)
    
    # Расчет метрик
    accuracy_train = accuracy_score(y_train, y_pred_train)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    roc_auc = roc_auc_score(y_test, y_pred_proba_test)
    f1 = f1_score(y_test, y_pred_test)
    precision = precision_score(y_test, y_pred_test)
    recall = recall_score(y_test, y_pred_test)
    conf_matrix = confusion_matrix(y_test, y_pred_test)
    
    # Вывод результатов
    print("Обучающая выборка:")
    print(f"Accuracy: {accuracy_train:.4f}")
    print("Тестовая выборка:")
    print(f"Accuracy: {accuracy_test:.4f}")
    print(f"ROC AUC: {roc_auc:.4f}")
    print(f"F1-score: {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print("Confusion Matrix:")
    print(conf_matrix)

In [47]:
from catboost import CatBoostClassifier

# Параметры для CatBoostClassifier
params = {
    'iterations': 200,              # Количество деревьев
    'learning_rate': 0.03,            # Скорость обучения
    'depth': 6,                      # Глубина дерева
    'l2_leaf_reg': 3,                # Коэффициент регуляризации L2 (lambda)
    'eval_metric': 'AUC',       # Метрика для оценки качества на валидации
    'random_seed': 666,              # Сид для воспроизводимости результатов
    'loss_function': 'Logloss',      # Функция потерь для задачи классификации
    'early_stopping_rounds': 50,     # Остановка обучения, если валидационная метрика не улучшается в течение заданного количества итераций
    'use_best_model': True,          # Использование лучшей модели после обучения
    'verbose': 20,                  # Частота вывода информации о процессе обучения
    'task_type': 'CPU'               # Тип задачи: 'CPU' или 'GPU', если у вас есть поддержка GPU
}

# Создание и обучение модели
model = CatBoostClassifier(**params)
model.fit(X_train, y_train, eval_set=(X_test, y_test))
evaluate_model(model, X_train, X_test, y_train, y_test)


0:	test: 0.9739011	best: 0.9739011 (0)	total: 5.19ms	remaining: 1.03s
20:	test: 0.9945055	best: 0.9948107 (15)	total: 150ms	remaining: 1.28s
40:	test: 0.9978632	best: 0.9978632 (38)	total: 232ms	remaining: 898ms
60:	test: 0.9978632	best: 0.9978632 (38)	total: 353ms	remaining: 805ms
80:	test: 0.9981685	best: 0.9981685 (68)	total: 445ms	remaining: 654ms
100:	test: 0.9984737	best: 0.9984737 (85)	total: 583ms	remaining: 571ms
120:	test: 0.9984737	best: 0.9984737 (85)	total: 684ms	remaining: 447ms
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9984737485
bestIteration = 85

Shrink model to first 86 iterations.
Обучающая выборка:
Accuracy: 1.0000
Тестовая выборка:
Accuracy: 0.9921
ROC AUC: 0.9985
F1-score: 0.9945
Precision: 0.9891
Recall: 1.0000
Confusion Matrix:
[[35  1]
 [ 0 91]]


In [48]:
model_path = 'catboost_model.cbm'
model.save_model(model_path, format="cbm")